## TASK 1

## PIPELINE OF THE NEWTROK: 

Taking ispiration from the most famous network we have, the main structure we can follow is that:
- (Stem layers)
- Resiudal blocks
- GlobalAvgPolling 
- FC layers 
For complexity reason (not so high capcity in terms of computation and memory) we prefer to not construct a network with so much layers (also to prevent overfit)

Network: what we can progressively augment:
- Adding stem layers -> max two layers
- Augment the number of channels of convolutional layers
- Augment the number of fully-connected layer
- Adding batch-norm layer
- Using residual block (as skip connection)
- Adding dropout
(In total I want that the newtork shows at most 10 layers)
(Using alway SiLu as activation function)

Hyperparamters: we can progressively use:
- Data augmentation
- Weight decacy 
- Larnenig schedule (also here we can see, based on how much high is the lr, what change)
- Varying dimension of batch size to see what is better (also looks for the task 2)

We fix the number of epoch as 50 with early stopping. 


Imports 

In [1]:
class BaseNet(nn.Module):
    def __init__(self, num_classes=43):
        super(BaseNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 112 * 112, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithStem(nn.Module):
    def __init__(self, num_classes=43):
        super(NetWithStem, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.SiLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.SiLU()
        )
        self.features = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithMoreChannels(nn.Module):
    def __init__(self, num_classes=43):
        super(NetWithMoreChannels, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.SiLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.SiLU()
        )
        self.features = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 56 * 56, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithMoreFC(nn.Module):
    def __init__(self, num_classes=43):
        super(NetWithMoreFC, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.SiLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.SiLU()
        )
        self.features = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 56 * 56, 512),
            nn.SiLU(),
            nn.Linear(512, 256),
            nn.SiLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithBatchNorm(nn.Module):
    def __init__(self, num_classes=43):
        super(NetWithBatchNorm, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU()
        )
        self.features = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 56 * 56, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithResidual(nn.Module):
    class ResidualBlock(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(NetWithResidual.ResidualBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(out_channels)
            self.shortcut = nn.Sequential()
            if in_channels != out_channels:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                    nn.BatchNorm2d(out_channels)
                )

        def forward(self, x):
            residual = x
            x = nn.SiLU()(self.bn1(self.conv1(x)))
            x = self.bn2(self.conv2(x))
            x += self.shortcut(residual)
            x = nn.SiLU()(x)
            return x

    def __init__(self, num_classes=43):
        super(NetWithResidual, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU()
        )
        self.features = nn.Sequential(
            self.ResidualBlock(64, 128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 56 * 56, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

class NetWithDropout(nn.Module):
    class ResidualBlock(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(NetWithDropout.ResidualBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(out_channels)
            self.shortcut = nn.Sequential()
            if in_channels != out_channels:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                    nn.BatchNorm2d(out_channels)
                )

        def forward(self, x):
            residual = x
            x = nn.LeakyReLU(negative_slope=0.1)(self.bn1(self.conv1(x)))
            x = self.bn2(self.conv2(x))
            x += self.shortcut(residual)
            x = nn.LeakyReLU(negative_slope=0.1)(x)
            return x

    def __init__(self, num_classes=43):
        super(NetWithDropout, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout2d(0.1),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout2d(0.1)
        )
        self.features = nn.Sequential(
            self.ResidualBlock(64, 128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            self.ResidualBlock(128, 256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.5)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 28 * 28, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

AttributeError: module 'torch' has no attribute 'cuda'